# Use O2C_Template Customized Notebook Template

In [1]:
import pandas as pd
import numpy as np

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [3]:
#table_name = 'ORDER_TO_CASH_ENRICHED'
table_name = 'ORDER_TO_CASH_MASTER'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [4]:
df['INVOICESTATUS'].value_counts()

INVOICESTATUS
LATE_PAYMENT       87656
ONTIME_PAYMENT     82789
Approved            1167
PendingApproval     1081
Sent                 744
Name: count, dtype: int64

In [5]:
df = df[df['INVOICESTATUS'] in ['LATE_PAYMENT','ONTIME_PAYMENT']]
df_new = df[df['INVOICESTATUS'] in ['Approved','PendingApproval','Sent']]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [114]:
df.shape, df_new.shape

((170290, 37), (0, 37))

In [115]:
# Assuming df is your dataframe
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'])
df['DELIVERYDATE'] = pd.to_datetime(df['DELIVERYDATE'])
df['INVOICEDATE'] = pd.to_datetime(df['INVOICEDATE'])
df['PAYMENTDATE'] = pd.to_datetime(df['PAYMENTDATE'])
df['INVOICEDUEDATE'] = pd.to_datetime(df['INVOICEDUEDATE'])
df['DELIVEREDON'] = pd.to_datetime(df['DELIVEREDON'])

In [116]:
# Total Sales Volume
total_sales_volume = df.groupby('PRODUCTID')['ORDERQUANTITY'].sum().reset_index(name='PR_TOTAL_SALES_VOLUME')

In [117]:
# Total Sales Value
total_sales_value = df.groupby('PRODUCTID')['ORDERVALUE'].sum().reset_index(name='PR_TOTAL_SALES_VALUE')

In [118]:
# Average Order Quantity
average_order_quantity = df.groupby('PRODUCTID')['ORDERQUANTITY'].mean().reset_index(name='PR_AVG_ORDER_QUANTITY')

In [119]:
# Average Unit Price
average_unit_price = df.groupby('PRODUCTID')['UNITPRICE'].mean().reset_index(name='PR_AVG_UNIT_PRICE')

In [120]:
# Number of Orders
number_of_orders = df.groupby('PRODUCTID')['ORDERID'].nunique().reset_index(name='PR_NUMBER_OF_ORDERS')

In [121]:
category_sales_volume = df.groupby('PRODUCTCATEGORY')['ORDERQUANTITY'].sum().reset_index(name='CAT_TOTAL_SALES_VOLUME')
category_sales_value = df.groupby('PRODUCTCATEGORY')['ORDERVALUE'].sum().reset_index(name='CAT_TOTAL_SALES_VALUE')
category_average_order_quantity = df.groupby('PRODUCTCATEGORY')['ORDERQUANTITY'].mean().reset_index(name='CAT_AVG_ORDER_QUANTITY')
category_average_unit_price = df.groupby('PRODUCTCATEGORY')['UNITPRICE'].mean().reset_index(name='CAT_AVG_UNIT_PRICE')
category_number_of_orders = df.groupby('PRODUCTCATEGORY')['ORDERID'].nunique().reset_index(name='CAT_NUMBER_OF_ORDERS')

In [122]:
delivery_performance.head()

,PRODUCTID,PR_AVG_DELIVERY_TIME
0,PID-064e463d-7de2-47bb-80f8-02b6fda2392a,8.454769
1,PID-08d031fd-c61f-47f3-b4d1-e9c79ba23b68,8.474123
2,PID-0952d502-dd95-42ff-8668-7551b8ba2d62,8.518086
3,PID-0dcaa3ac-5e13-4b31-b42a-70b594207dab,8.501065
4,PID-1194cfe3-dea0-496c-b865-3c38d22371cc,8.436611


In [123]:
# Delivery Performance
df['DELIVEREDON'] = pd.to_datetime(df['DELIVEREDON'])
df['PR_DELIVERY_TIME'] = (df['DELIVEREDON'] - df['ORDERDATE']).dt.days
delivery_performance = df.groupby('PRODUCTID')['PR_DELIVERY_TIME'].mean().reset_index(name='PR_AVG_DELIVERY_TIME')
category_delivery_performance = df.groupby('PRODUCTCATEGORY')['DELIVERY_TIME'].mean().reset_index(name='CAT_AVG_DELIVERY_TIME')

In [124]:
# Invoice Performance
df['PAYMENTDATE'] = pd.to_datetime(df['PAYMENTDATE'])
df['PR_INVOICE_TIME'] = (df['PAYMENTDATE'] - df['INVOICEDATE']).dt.days
invoice_performance = df.groupby('PRODUCTID')['PR_INVOICE_TIME'].mean().reset_index(name='PR_AVG_INVOICE_TIME')
category_invoice_performance = df.groupby('PRODUCTCATEGORY')['INVOICE_TIME'].mean().reset_index(name='CAT_AVG_INVOICE_TIME')

In [125]:
# Merge all KPIs into a single dataframe for PRODUCTID
product_kpis = total_sales_volume.merge(total_sales_value, on='PRODUCTID') \
                                 .merge(average_order_quantity, on='PRODUCTID') \
                                 .merge(average_unit_price, on='PRODUCTID') \
                                 .merge(number_of_orders, on='PRODUCTID') \
                                 .merge(delivery_performance, on='PRODUCTID') \
                                 .merge(invoice_performance, on='PRODUCTID')

In [126]:
# Merge all KPIs into a single dataframe for PRODUCTID
category_kpis = category_sales_volume.merge(category_sales_value, on='PRODUCTCATEGORY') \
                                 .merge(category_average_order_quantity, on='PRODUCTCATEGORY') \
                                 .merge(category_average_unit_price, on='PRODUCTCATEGORY') \
                                 .merge(category_number_of_orders, on='PRODUCTCATEGORY') \
                                 .merge(category_delivery_performance, on='PRODUCTCATEGORY') \
                                 .merge(category_invoice_performance, on='PRODUCTCATEGORY')

In [127]:
product_kpis.shape

(62, 8)

In [128]:
category_kpis.shape

(7, 8)

In [129]:
# Merge the new features back into the original dataframe
df = df.merge(product_kpis, on='PRODUCTID', how='left')

In [130]:
# Merge the new features back into the original dataframe
df = df.merge(category_kpis, on='PRODUCTCATEGORY', how='left')

In [131]:
df.columns

Index(['ORDERID', 'ORDERITEMID', 'PRODUCTID', 'PRODUCTNAME', 'ORDERQUANTITY',
       'UNITPRICE', 'ORDERVALUE', 'CUSTOMERID', 'CUSTOMERNAME',
       'PRODUCTCATEGORY', 'ORDERDATE', 'ORDERSTATUS', 'ORDERAMOUNT',
       'ORDERCOUNT', 'SHIPMENTID', 'SHIPMENTDATE', 'DELIVERYDATE',
       'DELIVEREDON', 'CARRIER', 'SHIPMENTSTATUS', 'INVOICEID', 'INVOICEDATE',
       'INVOICEDUEDATE', 'PAYMENTDATE', 'INVOICESTATUS', 'PAYMENTMETHOD',
       'PRODUCTTYPE', 'QUANTITY', 'NAME', 'COMPANYTYPE', 'CONTACTDETAILS',
       'EMAILDETAILS', 'ADDRESSDETAILS', 'ADMINDETAILS', 'CREDITLIMIT',
       'CREDITRATING', 'CUSTOMERTYPE', 'DELIVERY_TIME', 'INVOICE_TIME',
       'PR_TOTAL_SALES_VOLUME', 'PR_TOTAL_SALES_VALUE',
       'PR_AVG_ORDER_QUANTITY', 'PR_AVG_UNIT_PRICE', 'PR_NUMBER_OF_ORDERS',
       'PR_AVG_DELIVERY_TIME', 'PR_AVG_INVOICE_TIME', 'CAT_TOTAL_SALES_VOLUME',
       'CAT_TOTAL_SALES_VALUE', 'CAT_AVG_ORDER_QUANTITY', 'CAT_AVG_UNIT_PRICE',
       'CAT_NUMBER_OF_ORDERS', 'CAT_AVG_DELIVERY_TIME',
  

In [132]:
df = df[['PRODUCTID', 'PRODUCTNAME','PRODUCTCATEGORY','PRODUCTTYPE','UNITPRICE','QUANTITY',
         'PR_TOTAL_SALES_VOLUME', 'PR_TOTAL_SALES_VALUE','PR_AVG_ORDER_QUANTITY', 'PR_AVG_UNIT_PRICE', 'PR_NUMBER_OF_ORDERS',
         'PR_AVG_DELIVERY_TIME', 'PR_AVG_INVOICE_TIME', 
         'CAT_TOTAL_SALES_VOLUME','CAT_TOTAL_SALES_VALUE', 'CAT_AVG_ORDER_QUANTITY', 'CAT_AVG_UNIT_PRICE',
         'CAT_NUMBER_OF_ORDERS', 'CAT_AVG_DELIVERY_TIME','CAT_AVG_INVOICE_TIME']]

In [133]:
df.head()

,PRODUCTID,PRODUCTNAME,PRODUCTCATEGORY,PRODUCTTYPE,UNITPRICE,QUANTITY,PR_TOTAL_SALES_VOLUME,PR_TOTAL_SALES_VALUE,PR_AVG_ORDER_QUANTITY,PR_AVG_UNIT_PRICE,PR_NUMBER_OF_ORDERS,PR_AVG_DELIVERY_TIME,PR_AVG_INVOICE_TIME,CAT_TOTAL_SALES_VOLUME,CAT_TOTAL_SALES_VALUE,CAT_AVG_ORDER_QUANTITY,CAT_AVG_UNIT_PRICE,CAT_NUMBER_OF_ORDERS,CAT_AVG_DELIVERY_TIME,CAT_AVG_INVOICE_TIME
0,PID-4b87db26-edd8-4b4a-8143-adb85ba6dbe2,Insulation Material,Construction Materials,Insulation Material,2,10000,18260514,36521028,5024.907540,2.0,3634,8.531921,8.058888,39088690,1304027584,2158.761253,142.177390,9053,8.502844,8.032308
1,PID-89334032-db8d-4821-a9ad-5bd9a5305740,Cement,Construction Materials,Cement,100,500,912359,91235900,250.372942,100.0,3644,8.521679,8.071350,39088690,1304027584,2158.761253,142.177390,9053,8.502844,8.032308
2,PID-3e2ce142-a70d-4da4-8770-2079f75f43a9,Roofing Sheets,Construction Materials,Roofing Sheets,7,5000,9063971,63447797,2503.167909,7.0,3621,8.514499,8.038663,39088690,1304027584,2158.761253,142.177390,9053,8.502844,8.032308
3,PID-fec6ed6f-7ac1-4655-b4a7-9766cff15b7b,Engine Oil,Automotive,Engine Oil,40,1000,828858,33154320,507.878676,40.0,1632,8.404412,7.985907,13100980,443738184,829.438430,38.239696,7892,8.497752,7.915543
4,PID-8b2ea2ce-e1df-4139-b179-3e7f217a9f9b,Conveyor Belt,Industrial Equipment,Conveyor Belt,1000,1000,679669,679669000,504.579807,1000.0,1347,8.550854,7.939866,1316525,13397349740,94.415161,18058.703385,6945,8.492470,8.043890


In [134]:
df.shape

(170290, 20)

In [135]:
df = df.drop_duplicates()

In [136]:
df.shape

(62, 20)

In [137]:
df_train_sf=my_session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_HORIZONTAL.O2C_GOLD.ORDER_TO_CASH_PRODUCT_KPI")